In [1]:
import tensorflow as tf
import os
import pickle as p
import numpy as np 

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['data_batch_5', 'batches.meta', 'data_batch_4', 'data_batch_3', 'data_batch_1', 'readme.html', 'data_batch_2', 'test_batch']


In [2]:
def load_data(filename):
    """read data from data file"""
    with open(filename,'rb') as f:
        data = p.load(f,encoding='bytes')
        return data[b'data'],data[b'labels']
    
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)  #纵向合并成一个矩阵
        self._labels = np.hstack(all_labels)  #横向合并～
        print(self._data.shape)
        print(self._labels.shape)
  
        self._num_examples = self._data.shape[0]  #样本数目
        self._need_shuffle = need_shuffle   
        #样本数据是否需要打乱，是否可以重复使用数据样本
        self._indicator = 0  #表明当前数据集已经定位到哪个位置了
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        #顺序打乱  将0～(num-1)的数混乱排序并输出
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self,batch_size):
        """return batch_size example as a batch"""
        end_indicator = self._indicator + batch_size #定义结束位置
        if end_indicator > self._num_examples:  
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more example")
        if end_indicator > self._num_examples:  
        #这个判断batch_size是不是大于样本总数
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator:end_indicator]
        batch_labels = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR,'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR,'test_batch')]

train_data = CifarData(train_filenames,True)
test_data = CifarData(test_filenames,False)

# batch_data, batch_labels = train_data.next_batch(10)
# print(batch_data)
# print(batch_labels)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
def conv_wrapper(inputs,
                 output_channel,
                 strides,
                 name,
                 is_training,
                 kernel_size = (3, 3),
                 padding = 'same',
                 activation = tf.nn.relu):
    with tf.name_scope(name):
        conv2d = tf.layers.conv2d(inputs,
                                  output_channel,
                                  kernel_size,
                                  strides = strides,
                                  padding = padding,
                                  activation = None,
                                  name = name + '/conv2d')
        bn = tf.layers.batch_normalization(conv2d,training = is_training)
        return activation(bn)


# 残差连接块
def residual_block(x, output_channel, is_training):
    """residual connection implementation"""
    input_channel = x.get_shape().as_list()[-1]
    if input_channel * 2 == output_channel:
        increase_dim = True,
        strides = (2, 2)
    elif input_channel == output_channel:
        increase_dim = False
        strides = (1, 1)
    else:
        raise Exception("input channel can't match output channel")
        
    conv1 = conv_wrapper(x, output_channel, strides, 'conv1', is_training)
    conv2 = conv_wrapper(conv1, output_channel, (1,1), 'conv2', is_training)
    conv3 = conv_wrapper(conv2, output_channel, (1,1), 'conv3', is_training)
#     conv1 = tf.layers.conv2d(x,
#                             output_channel,
#                             (3, 3),
#                             strides = strides,
#                             padding = 'same',
#                             activation = tf.nn.relu,
#                             name = 'conv1')
#     conv2 = tf.layers.conv2d(conv1,
#                             output_channel,
#                             (3, 3),
#                             strides = (1, 1),
#                             padding = 'same',
#                             activation = tf.nn.relu,
#                             name = 'conv2')
    if increase_dim:
        # [None, image_height, image_width, channel] -> [,,,channel*2]
        pooled_x =tf.layers.average_pooling2d(x,
                                             (2, 2),
                                             (2, 2),
                                             padding = 'valid')
        padded_x = tf.pad(pooled_x,
                         [[0, 0],
                          [0, 0],
                          [0, 0],
                          [input_channel // 2, output_channel - input_channel - input_channel //2]])
    else:
        padded_x = x
    output_x = conv2 + padded_x
    return output_x

def res_net(x,
            num_residual_blocks, #每一层需要多少残差连接块
            num_filter_base, #最初通道数目
            class_num,
            is_training): #类别数目
    """residual network implementation"""
    """
    Args:
    - x:
    - num_residual_blocks: eg:[3, 4, 6, 3]
    - num_filter_base:
    - class_num:
    """
    # 多少次降采样
    num_subsampling = len(num_residual_blocks)
    layers = []
    # x:[None, height, width, channel] -> [height,width,  channel]
    input_size = x.get_shape().as_list()[1:]
    with tf.variable_scope('conv0'):
        conv0 = conv_wrapper(x, num_filter_base, (1,1), 'conv0', is_training)
#         conv0 = tf.layers.conv2d(x,
#                                  num_filter_base,
#                                  (3,3),
#                                  strides = (1, 1),
#                                  padding = 'same',
#                                  activation = tf.nn.relu,
#                                  name = 'conv0')
        layers.append(conv0)
    for sample_id in range(num_subsampling):
        for i in range(num_residual_blocks[sample_id]):
            with tf.variable_scope("conv%d_%d" % (sample_id, i)):
                conv = residual_block(
                    layers[-1],
                    num_filter_base * (2 ** sample_id),
                    is_training)
                layers.append(conv)
    multiplier = 2 ** (num_subsampling - 1)
    assert layers[-1].get_shape().as_list()[1:] \
        == [input_size[0] / multiplier,
            input_size[1] / multiplier,
            num_filter_base * multiplier],"equal error"
    with tf.variable_scope('fc'):
        # layers[-1].shape : [None, width, height, channel]
        #kernal_size: (image_width, image_height)
        global_pool = tf.reduce_mean(layers[-1], [1, 2])
        # logits 是做softmax之前的值
        logits = tf.layers.dense(global_pool,class_num)
        layers.append(logits)
    return layers[-1]

batch_size = 20
x = tf.placeholder(tf.float32,[batch_size,3072])
y = tf.placeholder(tf.int64,[batch_size])
is_training = tf.placeholder(tf.bool, [])

x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

result_x_image_arr = []
x_single_image_arr = tf.split(x_image, num_or_size_splits=batch_size, axis=0)
for x_single_image in x_single_image_arr:
    x_single_image = tf.reshape(x_single_image, [32,32,3])
    data_aug_1 = tf.image.random_flip_left_right(x_single_image)
    data_aug_2 = tf.image.random_brightness(data_aug_1, max_delta=70)
    data_aug_3 = tf.image.random_contrast(data_aug_2, lower=0.3, upper=1.9)
    x_single_image = tf.reshape(data_aug_3, [1,32,32,3])
    result_x_image_arr.append(x_single_image)
result_x_images = tf.concat(result_x_image_arr, axis=0)
normal_result_x_images = result_x_images / 127.5 - 1

y_ = res_net(normal_result_x_images, [2,2,2,2], 64, 10, is_training)

# 交叉熵损失
loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)
# y_ -> softmax
#y -> one_hot
#loss = ylogy_


# indices
predict = tf.argmax(y_, 1) #在y_的第一维求最大值的索引位置
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    optimizer = tf.train.AdamOptimizer(1e-3)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss)

In [4]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

# train 10000: 74.7%
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data,batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict = {
                x: batch_data, 
                y: batch_labels,
                is_training: True,})
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' \
                 % (i+1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames,False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels,
                        is_training: False,
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' % (i+1,test_acc))

[Train] Step: 500, loss: 2.06299, acc: 0.25000
[Train] Step: 1000, loss: 1.91688, acc: 0.25000
[Train] Step: 1500, loss: 1.07713, acc: 0.70000
[Train] Step: 2000, loss: 1.46896, acc: 0.45000
[Train] Step: 2500, loss: 1.24384, acc: 0.55000
[Train] Step: 3000, loss: 1.22793, acc: 0.55000
[Train] Step: 3500, loss: 0.90722, acc: 0.75000
[Train] Step: 4000, loss: 0.83070, acc: 0.80000
[Train] Step: 4500, loss: 0.81870, acc: 0.75000
[Train] Step: 5000, loss: 0.65321, acc: 0.75000
(10000, 3072)
(10000,)
[Test] Step: 5000, acc: 0.65200


KeyboardInterrupt: 